In [6]:
import pandas as pd

# Пути к файлам


# Загрузка данных
financial_data = pd.read_csv("financial_data.csv")
prolongations = pd.read_csv("prolongations.csv")

# Приведение названий колонок к единому виду
financial_data.rename(columns={'Account': 'AM'}, inplace=True)

# Приведение названий месяцев к единому формату
month_mapping = {
    'Ноябрь 2022': 'Ноябрь', 'Декабрь 2022': 'Декабрь', 'Январь 2023': 'Январь',
    'Февраль 2023': 'Февраль', 'Март 2023': 'Март', 'Апрель 2023': 'Апрель',
    'Май 2023': 'Май', 'Июнь 2023': 'Июнь', 'Июль 2023': 'Июль',
    'Август 2023': 'Август', 'Сентябрь 2023': 'Сентябрь', 'Октябрь 2023': 'Октябрь',
    'Ноябрь 2023': 'Ноябрь', 'Декабрь 2023': 'Декабрь', 'Январь 2024': 'Январь',
    'Февраль 2024': 'Февраль'
}
financial_data.rename(columns=month_mapping, inplace=True)

# Приведение `month` в prolongations к нужному формату
prolongations['month'] = prolongations['month'].astype(str).str.capitalize().str.replace(r' \d{4}', '', regex=True)

# Конвертация сумм в числа
for month in month_mapping.values():
    if month in financial_data.columns:
        financial_data[month] = pd.to_numeric(
            financial_data[month].astype(str).str.replace('\u00a0', '').str.replace(',', '.'), errors='coerce'
        )

# Объединение данных по ID проекта
merged_data = prolongations.merge(financial_data, on=['id', 'AM'], how='left')

# Список месяцев для расчета
months = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
          'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']

# Функция расчета коэффициентов пролонгации
def calculate_prolongation_coefficients(data, months):
    results = []
    for i in range(len(months) - 2):
        prev_month, first_month, second_month = months[i], months[i+1], months[i+2]
        
        # Проекты, завершившиеся в предыдущем месяце
        completed_projects = data[data['month'] == prev_month]
        total_prev_month = completed_projects[prev_month].sum()
        
        # Пролонгация в первый месяц
        first_prolongation = completed_projects.loc[completed_projects[first_month] > 0, first_month].sum()
        first_ratio = first_prolongation / total_prev_month if total_prev_month > 0 else 0
        
        # Проекты, не пролонгированные в первый месяц
        non_prolonged = completed_projects.loc[completed_projects[first_month] == 0]
        total_non_prolonged = non_prolonged[prev_month].sum()
        
        # Пролонгация во второй месяц
        second_prolongation = non_prolonged.loc[non_prolonged[second_month] > 0, second_month].sum()
        second_ratio = second_prolongation / total_non_prolonged if total_non_prolonged > 0 else 0
        
        results.append({
            'Месяц': first_month,
            'Коэффициент пролонгации 1 мес': first_ratio,
            'Коэффициент пролонгации 2 мес': second_ratio
        })
    
    return pd.DataFrame(results)

# Расчет коэффициентов
prolongation_report = calculate_prolongation_coefficients(merged_data, months)

# Сохранение в Excel
prolongation_report.to_excel('prolongation_report.xlsx', index=False)
print("Отчет успешно создан!")


AttributeError: 'DataFrame' object has no attribute 'str'